In [29]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [30]:
memory = ConversationBufferMemory(return_messages=True)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

In [31]:
loader = TextLoader("./files/document.txt")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=650,
    chunk_overlap=100,
)
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cache_dir = LocalFileStore("./.cache/")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

In [32]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using the following context:\n\n{context}\n\nand You refer to history:"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

chain =  {"context": retriver, "question": RunnablePassthrough()} | RunnablePassthrough.assign(history=load_memory) | prompt | llm

In [33]:
def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )

    return result.content

In [34]:
invoke_chain("Is Aaronson guilty?")

According to the document, Jones, Aaronson, and Rutherford were considered guilty of the crimes they were charged with. However, it was later revealed that the photograph disproving their guilt never existed and was invented.

'According to the document, Jones, Aaronson, and Rutherford were considered guilty of the crimes they were charged with. However, it was later revealed that the photograph disproving their guilt never existed and was invented.'

In [35]:
invoke_chain("What message did he write in the table?")

He wrote the message "FREEDOM IS SLAVERY" in large clumsy capitals on the table.

'He wrote the message "FREEDOM IS SLAVERY" in large clumsy capitals on the table.'

In [36]:
invoke_chain("Who is Julia?")

Julia is a character in the document who is involved in a romantic relationship with the protagonist. She is also a member of the Party and plays a significant role in the story.

'Julia is a character in the document who is involved in a romantic relationship with the protagonist. She is also a member of the Party and plays a significant role in the story.'